In [5]:
import hoomd
import gsd.hoomd
import numpy as np

In [14]:
rigid_gsd = gsd.hoomd.open("/home/marjanalbooyeh/Aniso_ML_MD_project/Aniso_MLMD/aniso_MLMD/sampling/assets/pps_rigid_initial_run.gsd")

init_snap = hoomd.Snapshot()
# Create place holder spots in the snapshot for rigid centers
init_snap.particles.types =["A"]
init_snap.particles.N = 2
init_snap.particles.position[:] = rigid_gsd[0].particles.position[0:2]
init_snap.particles.orientation[:] = rigid_gsd[0].particles.orientation[0:2]
init_snap.particles.mass[:] = rigid_gsd[0].particles.mass[0:2]
init_snap.particles.moment_inertia[:] = rigid_gsd[0].particles.moment_inertia[0:2]
init_snap.configuration.box = rigid_gsd[0].configuration.box

In [120]:
import signac

project = signac.get_project("/home/marjanalbooyeh/Aniso_ML_MD_project/Aniso_MLMD/aniso_MLMD/train_flow/pps_pair_Jan9/")

In [121]:
best_job_id = "e6ae5bd975c95259f509f3d8bc732f4e"
for j in project.find_jobs():
    if j.id == best_job_id:
        break

In [122]:
best_job = j

In [123]:
best_job.sp

{'project': 'pps-pair-Jan9', 'group': 'batch2', 'notes': 'Learning pps forces and torques', 'tags': 'w/o initialization', 'data_path': '/home/marjanalbooyeh/Aniso_ML_MD_project/ml_datasets/pps_pair/', 'batch_size': 64, 'shrink': False, 'overfit': False, 'in_dim': 92, 'hidden_dim': 64, 'n_layer': 5, 'act_fn': 'Tanh', 'dropout': 0.01, 'batch_norm': False, 'optim': 'Adam', 'lr': 0.001, 'min_lr': 0.0001, 'use_scheduler': True, 'scheduler_type': 'ReduceLROnPlateau', 'decay': 0.0001, 'loss_type': 'mse', 'prior_energy': True, 'prior_energy_sigma': 1, 'prior_energy_n': 12, 'epochs': 5000}

In [265]:
import hoomd
import torch
import numpy as np
import rowan
from aniso_MLMD.model import PairNN_Force_Torque
from aniso_MLMD.utils import _calculate_torque, _calculate_prior_energy

class PairEllipsCustomForce(hoomd.md.force.Custom):
    def __init__(self, model_path, in_dim, hidden_dim,
                 n_layers, act_fn, dropout, prior_energy, prior_energy_sigma, prior_energy_n):
        super().__init__(aniso=True)
        # load ML model
        self.ellipse_ids = [0, 1]
        self.device = torch.device(
            "cuda:0" if torch.cuda.is_available() else "cpu")
        self.model = PairNN_Force_Torque(in_dim=in_dim, hidden_dim=hidden_dim,
                        n_layers=n_layers, act_fn=act_fn, dropout=dropout, batch_norm=False, device=self.device)
        self.model.to(self.device)
        self.model.load_state_dict(
            torch.load(model_path, map_location=self.device)["model"])
        self.model.to(self.device)
        self.model.eval()
        self.prior_energy = prior_energy
        self.prior_energy_sigma = prior_energy_sigma
        self.prior_energy_n = prior_energy_n
        self.print_counter = 0

    def set_forces(self, timestep):
        self.print_counter += 1
        # get positions and orientations
        with self._state.cpu_local_snapshot as snap:
            rtags = snap.particles.rtag[self.ellipse_ids]
            positions = np.array(snap.particles.position[rtags],
                                 copy=True)
            orientations = np.array(snap.particles.orientation[rtags],
                                    copy=True)

        orientation_rotation_matrix = rowan.to_matrix(orientations)

        x1 = torch.from_numpy(positions[0, :]).type(torch.FloatTensor).unsqueeze(0)
        x2 = torch.from_numpy(positions[1, :]).type(torch.FloatTensor).unsqueeze(0)
        R1 = torch.from_numpy(orientation_rotation_matrix[0, :]).type(
            torch.FloatTensor).unsqueeze(0)
        R2 = torch.from_numpy(orientation_rotation_matrix[1, :]).type(
            torch.FloatTensor).unsqueeze(0)
        x1.requires_grad = True
        R1.requires_grad = True
        energy_prediction = self.model(x1, x2, R1, R2)

        if self.prior_energy:
            prior_energy = _calculate_prior_energy(x1, x2, self.prior_energy_sigma, self.prior_energy_n).to(self.device)
            energy_prediction += prior_energy
        predicted_force = - torch.autograd.grad(energy_prediction.sum(),
                                                x1,
                                                create_graph=True)[0].to(self.device)

        torque_grad = - torch.autograd.grad(energy_prediction.sum(),
                                            R1,
                                            create_graph=True)[0]
        predicted_torque = _calculate_torque(torque_grad, R1).to(self.device)

        torque_value = predicted_torque.cpu().detach().numpy()[0]
        force_value = predicted_force.cpu().detach().numpy()[0]

        if self.print_counter % 100000 == 0:
            print("*****************************")
            print("energy: ", energy_prediction)
            print("force: ", force_value)
            print("torque: ", torque_value)
        with self.cpu_local_force_arrays as arrays:
            #             print('timestep: ', timestep)
            #             print('****************************************')
            #             print(arrays.potential_energy)
            arrays.force[rtags] = force_value
            arrays.torque[rtags] = torque_value

In [266]:
device = hoomd.device.CPU()
sim = hoomd.Simulation(device=device)

sim.create_state_from_snapshot(init_snap)
ff = []
custom_force = PairEllipsCustomForce(model_path=best_job.fn("best_model_checkpoint.pth"),
                                     in_dim=best_job.sp.in_dim,
                                     hidden_dim=best_job.sp.hidden_dim,
                                     n_layers=best_job.sp.n_layer,
                                     act_fn=best_job.sp.act_fn,
                                     dropout=best_job.sp.dropout,
                                     prior_energy=best_job.sp.prior_energy,
                                     prior_energy_sigma=best_job.sp.prior_energy_sigma,
                                     prior_energy_n=best_job.sp.prior_energy_n)
ff.append(custom_force)

In [267]:
dt=0.005
kT=1.5
_all = hoomd.filter.All()
nvt = hoomd.md.methods.ConstantVolume(
    filter=_all,
    thermostat=hoomd.md.methods.thermostats.Bussi(kT=kT))
integrator = hoomd.md.Integrator(dt=dt, integrate_rotational_dof=True)
integrator.methods.append(nvt)
integrator.forces = ff
sim.operations.add(integrator)

In [268]:
sim.state.thermalize_particle_momenta(filter=_all, kT=kT)

*Warning*: Simulation.seed is not set, using default seed=0


In [269]:
thermodynamic_properties = hoomd.md.compute.ThermodynamicQuantities(filter=_all)
sim.operations.computes.append(thermodynamic_properties)

In [270]:
log_quantities = [
                    "kinetic_temperature",
                    "potential_energy",
                    "kinetic_energy",
                    "volume",
                    "pressure",
                    "pressure_tensor",
                ]
logger = hoomd.logging.Logger(categories=["scalar", "string", "particle"])
logger.add(sim, quantities=["timestep", "tps"])
thermo_props = hoomd.md.compute.ThermodynamicQuantities(filter=hoomd.filter.All())
sim.operations.computes.append(thermo_props)
logger.add(thermo_props, quantities=log_quantities)


gsd_writer = hoomd.write.GSD(
    filename="ML_sim_traj.gsd",
    trigger=hoomd.trigger.Periodic(int(10)),
    mode="wb",
    logger=logger,
    filter=hoomd.filter.All(),
    dynamic=["momentum", "property"]
    )

sim.operations.writers.append(gsd_writer)

In [271]:
init_snap.particles.position

array([[-3.27664495, -1.2271961 , -3.80518508],
       [ 1.44320226, -1.3552891 , -2.81317234]])

In [272]:
sim.run(0, write_at_start=True)

*Warning*: Simulation.seed is not set, using default seed=0


In [273]:
sim.state.get_snapshot().particles.position

array([[-3.27664495, -1.2271961 , -3.80518508],
       [ 1.44320226, -1.3552891 , -2.81317234]])

In [274]:
sim.run(1e5)

*****************************
energy:  tensor([[4.1497]], device='cuda:0', grad_fn=<AddBackward0>)
force:  [-0.24583215 -0.30942142  0.08680633]
torque:  [-0.7204759  -0.12738574  4.1768775 ]


In [275]:
sim.state.get_snapshot().particles.position

array([[-5.32870869, -2.49912469, -4.88227171],
       [ 6.47956284,  3.58321504,  2.44265436]])

In [276]:
sim.timestep

100000

In [277]:
sim.operations.writers[0].flush()

In [278]:
import gsd.hoomd
with gsd.hoomd.open("Ellipse_ML_sim_traj.gsd", "w") as new_t:
    with gsd.hoomd.open("ML_sim_traj.gsd", "r") as old_t:
        for snap in old_t:
            snap.particles.type_shapes = [
                {"type": "Ellipsoid",
                    "a": 1,
                    "b": 0.5,
                    "c":0.5}
            ]
            snap.validate()
            new_t.append(snap)

In [281]:
traj = gsd.hoomd.open("ML_sim_traj.gsd")

In [282]:
for i, frame in enumerate(traj):
    print(i)
    print(frame.particles.position)

0
[[-3.276645  -1.2271961 -3.805185 ]
 [ 1.4432023 -1.3552891 -2.8131723]]
1
[[-3.268814  -1.2067933 -3.8223631]
 [ 1.4365195 -1.3755946 -2.7961574]]
2
[[-3.258532  -1.1819751 -3.8433416]
 [ 1.4291979 -1.4001516 -2.775663 ]]
3
[[-3.245993  -1.1530019 -3.8679576]
 [ 1.4212153 -1.4287379 -2.7519317]]
4
[[-3.2330627 -1.1235032 -3.8930752]
 [ 1.4132613 -1.457816  -2.7278476]]
5
[[-3.2226093 -1.0997609 -3.9132977]
 [ 1.406912  -1.481202  -2.7084842]]
6
[[-3.2085996 -1.0680426 -3.9402988]
 [ 1.398483  -1.5124207 -2.6826217]]
7
[[-3.1947649 -1.0367916 -3.9668903]
 [ 1.3902158 -1.543162  -2.6571424]]
8
[[-3.1837533 -1.0119689 -3.9880018]
 [ 1.3836777 -1.5675622 -2.636909 ]]
9
[[-3.169479  -0.9798967 -4.0152464]
 [ 1.3752929 -1.5990454 -2.6107697]]
10
[[-3.1580184  -0.95422155 -4.03703   ]
 [ 1.3686318  -1.624198   -2.5898612 ]]
11
[[-3.148196  -0.9323091 -4.055604 ]
 [ 1.3629997 -1.6456336 -2.5720246]]
12
[[-3.1354291 -0.9039194 -4.079644 ]
 [ 1.3557545 -1.6733733 -2.5489185]]
13
[[-3.1202445 

In [280]:
sim.timestep

100000